In [ ]:
## Install NeMo
BRANCH = 'v1.8.2'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install jiwer
!pip install noisereduce

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml --no-check-certificate

  Cloning https://github.com/NVIDIA/NeMo.git (to revision v1.8.2) to /tmp/pip-install-jxy1814n/nemo-toolkit_faee7d05614f47d1b10f616774722097
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-jxy1814n/nemo-toolkit_faee7d05614f47d1b10f616774722097
  Running command git checkout -q 77be76349901d0dd99ad6778509ef5c0663a65c3
     |████████████████████████████████| 952 kB 5.4 MB/s 
     |████████████████████████████████| 12.8 MB 30.3 MB/s 
     |████████████████████████████████| 109 kB 35.2 MB/s 
     |████████████████████████████████| 99 kB 448 kB/s 
     |████████████████████████████████| 235 kB 47.6 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 42 kB 968 kB/s 
  Using cached pytest_runner-6.0.0-py3-none-any.whl (7.2 kB)
     |████████████████████████████████| 1.8 MB 31.7 MB/s 
     |████████████████████████████████| 165.5 MB 55 kB/s 
     |███████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
from omegaconf import DictConfig
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile

[NeMo W 2022-05-20 10:13:23 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
zip_dir = '/content/drive/My Drive/location_data.zip'
unzip_dir = '/content/drive/My Drive/location_data'
data_dir = '/content/drive/My Drive/'
if not os.path.exists(unzip_dir):
    zip_ref = zipfile.ZipFile(zip_dir, 'r')
    zip_ref.extractall(path=data_dir)

In [23]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data/audio_files_train/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data/train_script.txt'
train_manifest_path = '/home/dgxuser/Desktop/kailin/location_data/train_manifest.json'

audio_paths = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)

    rate, data = wavfile.read(audio_dir + str(n).zfill(4) + '_resampled.wav')
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(audio_dir + str(n).zfill(4) + '_clean.wav', rate, reduced_noise)
    audio_paths.append(audio_dir + str(n).zfill(4) + '_clean.wav')
    n = n+1

with open(train_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [18]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-05-20 11:02:18 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-05-20 11:02:18 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-05-20 11:02:18 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-05-20 11:02:19 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-05-20 11:02:19 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-05-20 11:02:19 features:259] PADDING: 16
[NeMo I 2022-05-20 11:02:19 features:276] STFT using torch
[NeMo I 2022-05-20 11:02:20 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [19]:
# test WER before tuning 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
speakers = ['0010']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

Word Error Rate: 0.04053471323846486
Number of Testcases (Lines): 380


In [24]:
# test before training 
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths: 
        line = script.readline().strip()
        files = [path]
        print(f'{line} was predicted as {quartznet.transcribe(paths2audio_files=files)[0]}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bishan bishan was predicted as be shan be shun


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

boon lay boon lay was predicted as good life bornlay


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

jurong jurong was predicted as juon jurone


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

tiong bahru tiong bahru was predicted as tunbaru tambury


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

yishun yishun was predicted as he shouldn't i soon


In [25]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [26]:
# Original optimizer + scheduler
from omegaconf import OmegaConf, open_dict
print(OmegaConf.to_yaml(quartznet.cfg.optim))

name: novograd
lr: 0.01
betas:
- 0.8
- 0.5
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: null
  warmup_ratio: null
  min_lr: 0.0
  last_epoch: -1



In [27]:
with open_dict(quartznet.cfg.optim):
  quartznet.cfg.optim.lr = 0.01
  quartznet.cfg.optim.betas = [0.95, 0.5]  # from paper
  quartznet.cfg.optim.weight_decay = 0.001  # Original weight decay
  quartznet.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  quartznet.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  quartznet.cfg.optim.sched.min_lr = 1e-5

In [28]:
print(OmegaConf.to_yaml(quartznet.cfg.optim))

name: novograd
lr: 0.01
betas:
- 0.95
- 0.5
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: null
  warmup_ratio: 0.05
  min_lr: 1.0e-05
  last_epoch: -1



In [29]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen, and batch normalization has been unfrozen")

Model encoder has been frozen, and batch normalization has been unfrozen


In [30]:
print(OmegaConf.to_yaml(quartznet.cfg.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
rect_freq: 50
rect_masks: 5
rect_time: 120



In [32]:
with open_dict(quartznet.cfg.spec_augment):
    quartznet.cfg.spec_augment.freq_masks = 2
    quartznet.cfg.spec_augment.freq_width = 25
    quartznet.cfg.spec_augment.time_masks = 2
    quartznet.cfg.spec_augment.time_width = 0.05

In [33]:
print(OmegaConf.to_yaml(quartznet.cfg.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
rect_freq: 50
rect_masks: 5
rect_time: 120
freq_masks: 2
freq_width: 25
time_masks: 2
time_width: 0.05



In [34]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data/test_script.txt'
val_manifest_path = '/home/dgxuser/Desktop/kailin/location_data/val_manifest.json'

audio_paths = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)

    rate, data = wavfile.read(audio_dir + str(n).zfill(4) + '_resampled.wav')
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(audio_dir + str(n).zfill(4) + '_clean.wav', rate, reduced_noise)
    audio_paths.append(audio_dir + str(n).zfill(4) + '_clean.wav')
    n = n+1

with open(val_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [35]:
# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = train_manifest_path

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=50, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-05-20 11:12:16 collections:186] Dataset loaded with 177 files totalling 0.15 hours
[NeMo I 2022-05-20 11:12:16 collections:187] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-20 11:12:16 collections:186] Dataset loaded with 177 files totalling 0.15 hours
[NeMo I 2022-05-20 11:12:16 collections:187] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-05-20 11:12:16 modelPT:475] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-05-20 11:12:16 modelPT:587] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-05-20 11:12:16 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
107 K     Trainable params
18.8 M    Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [36]:
test_paths = audio_paths[:10]
print(test_paths)

['/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0001_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0002_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0003_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0004_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0005_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0006_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0007_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0008_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0009_clean.wav', '/home/dgxuser/Desktop/kailin/location_data/audio_files_test/0010_clean.wav']


In [37]:
with open(script_dir, encoding = 'utf-8') as script:
    for path in test_paths: 
        line = script.readline().strip()
        files = [path]
        print(f'{line} was predicted as {quartznet.transcribe(paths2audio_files=files)[0]}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

I live in jurong was predicted as bulas tong jurong


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

I want to go to sembawang was predicted as batuan a isembawang


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

sembawang sembawang was predicted as sembawang sembawang


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

tiong bahru tiong bahru was predicted as tiong bahru tiong bahru


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

tuas tuas was predicted as tuas tuas


In [38]:
# test WER after tuning 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
speakers = ['0010']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

Word Error Rate: 0.9984907287623976
Number of Testcases (Lines): 380
